<a href="https://colab.research.google.com/github/himanib09/Machine-Learning/blob/master/FND03_product_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analyze Product Sentiment

In [ ]:
!pip install turicreate

     |████████████████████████████████| 91.8MB 74kB/s 
     |████████████████████████████████| 2.1MB 38.1MB/s 
     |████████████████████████████████| 20.2MB 1.4MB/s 
     |████████████████████████████████| 3.4MB 37.8MB/s 
     |████████████████████████████████| 86.3MB 46kB/s 
     |████████████████████████████████| 3.8MB 36.9MB/s 
     |████████████████████████████████| 450kB 49.3MB/s 
ERROR: fastai 0.7.0 has requirement torch<0.4, but you'll have torch 1.4.0 which is incompatible.
ERROR: tensorboard 2.0.2 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
  Found existing installation: Pillow 4.3.0
    Uninstalling Pillow-4.3.0:
      Successfully uninstalled Pillow-4.3.0
  Found existing installation: llvmlite 0.29.0
    Uninstalling llvmlite-0.29.0:
      Successfully uninstalled llvmlite-0.29.0
  Found existing installation: google-auth-oauthlib 0.4.0
    Uninstalling google-auth-oauthlib-0.4.0:
      Successfully uninstalled google-auth-oauthlib-

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# change the working directory of your code on google drive
%cd/content/drive/My Drive/Colab Notebooks

/content/drive/My Drive/Colab Notebooks


In [ ]:
import turicreate

# Read product review data

In [ ]:
products = turicreate.SFrame('data/amazon_baby.sframe')

# Explore data

In [ ]:
products

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


In [ ]:
products.groupby('name',operations={'count':turicreate.aggregate.COUNT()}).sort('count',ascending=False)

name,count
Vulli Sophie the GiraffeTeether ...,785
"Simple Wishes Hands-FreeBreastpump Bra, Pink, ...",562
Infant Optics DXR-5 2.4GHz Digital Video Baby ...,561
Baby Einstein Take AlongTunes ...,547
Cloud b TwilightConstellation Night ...,520
"Fisher-Price BoosterSeat, Blue/Green/Gray ...",489
Fisher-Price RainforestJumperoo ...,450
"Graco Nautilus 3-in-1 CarSeat, Matrix ...",419
Leachco Snoogle TotalBody Pillow ...,388
"Regalo Easy Step WalkThru Gate, White ...",374


# Examine the reivews for the most-reviewed product

In [ ]:
giraffe_reviews = products[products['name']=='Vulli Sophie the Giraffe Teether']

In [ ]:
giraffe_reviews

name,review,rating
Vulli Sophie the GiraffeTeether ...,He likes chewing on allthe parts especially the ...,5.0
Vulli Sophie the GiraffeTeether ...,My son loves this toy andfits great in the diaper ...,5.0
Vulli Sophie the GiraffeTeether ...,There really should be alarge warning on the ...,1.0
Vulli Sophie the GiraffeTeether ...,All the moms in my moms'group got Sophie for ...,5.0
Vulli Sophie the GiraffeTeether ...,I was a little skepticalon whether Sophie was ...,5.0
Vulli Sophie the GiraffeTeether ...,I have been reading aboutSophie and was going ...,5.0
Vulli Sophie the GiraffeTeether ...,My neice loves her sophieand has spent hours ...,5.0
Vulli Sophie the GiraffeTeether ...,What a friendly face!And those mesmerizing ...,5.0
Vulli Sophie the GiraffeTeether ...,We got this just for myson to chew on instea ...,5.0
Vulli Sophie the GiraffeTeether ...,"My baby seems to likethis toy, but I could ...",3.0


In [ ]:
len(giraffe_reviews)

785

In [ ]:
giraffe_reviews['rating'].show()

Materializing SArray

# Building a sentiment classifier

## Build word count vectors

In [ ]:
products['word_count'] = turicreate.text_analytics.count_words(products['review'])

In [ ]:
products

name,review,rating,word_count
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0,"{'and': 5.0, 'stink':1.0, 'months': 1.0, ..."
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3.0, 'love': 1.0,'it': 3.0, 'highly': ..."
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2.0, 'quilt':1.0, 'it': 1.0, ..."
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'and': 3.0, 'ingenious':1.0, 'love': 2.0, 'is': ..."
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2.0, 'this': 2.0,'all': 2.0, 'love': 1.0, ..."
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2.0, 'cute': 1.0,'help': 2.0, 'habit': ..."
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1.0, 'be': 1.0,'is': 1.0, 'bound': 1.0, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'and': 2.0, 'all': 1.0,'right': 1.0, 'able': ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1.0, 'fantastic':1.0, 'help': 1.0, 'gi ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'pre': 1.0, 'all': 1.0,'standarad': 1.0, ..."


# Define what is positive and negative sentiment

In [ ]:
products['rating'].show()

Materializing SArray

In [ ]:
#ignore all 3*  reviews
products = products[products['rating']!= 3]

In [ ]:
#positive sentiment = 4-star or 5-star reviews
products['sentiment'] = products['rating'] >= 4

In [ ]:
products

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3.0, 'love': 1.0,'it': 3.0, 'highly': ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2.0, 'quilt':1.0, 'it': 1.0, ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'and': 3.0, 'ingenious':1.0, 'love': 2.0, 'is': ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2.0, 'this': 2.0,'all': 2.0, 'love': 1.0, ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2.0, 'cute': 1.0,'help': 2.0, 'habit': ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1.0, 'be': 1.0,'is': 1.0, 'bound': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'and': 2.0, 'all': 1.0,'right': 1.0, 'able': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1.0, 'fantastic':1.0, 'help': 1.0, 'gi ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'pre': 1.0, 'all': 1.0,'standarad': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'all': 2.0, 'forget':1.0, 'just': 1.0, 'fo ...",1


In [ ]:
products['sentiment'].show()

Materializing SArray

# Train our sentiment classifier

In [ ]:
train_data,test_data = products.random_split(.8,seed=0)

In [ ]:
sentiment_model = turicreate.logistic_classifier.create(train_data,target='sentiment', features=['word_count'], validation_set=test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 1

...

| 2         | 11       | 2.669978  | 5.169349     | 0.938650          | 0.916466            |

| 3         | 12       | 2.669978  | 5.777048     | 0.927417          | 0.901814            |

| 4         | 14       | 1.326072  | 6.702576     | 0.945200          | 0.918088            |

| 9         | 20       | 1.326072  | 10.024030    | 0.977654          | 0.917698            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

# Apply the sentiment classifier to better understand the Giraffe reviews

In [ ]:
products['predicted_sentiment'] = sentiment_model.predict(products, output_type = 'probability')

In [ ]:
products

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3.0, 'love': 1.0,'it': 3.0, 'highly': ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2.0, 'quilt':1.0, 'it': 1.0, ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'and': 3.0, 'ingenious':1.0, 'love': 2.0, 'is': ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2.0, 'this': 2.0,'all': 2.0, 'love': 1.0, ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2.0, 'cute': 1.0,'help': 2.0, 'habit': ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1.0, 'be': 1.0,'is': 1.0, 'bound': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'and': 2.0, 'all': 1.0,'right': 1.0, 'able': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1.0, 'fantastic':1.0, 'help': 1.0, 'gi ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'pre': 1.0, 'all': 1.0,'standarad': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'all': 2.0, 'forget':1.0, 'just': 1.0, 'fo ...",1


In [ ]:
giraffe_reviews = products[products['name']== 'Vulli Sophie the Giraffe Teether']

In [ ]:
giraffe_reviews

name,review,rating,word_count,sentiment
Vulli Sophie the GiraffeTeether ...,He likes chewing on allthe parts especially the ...,5.0,"{'and': 1.0, 'all': 1.0,'because': 1.0, 'it': ...",1
Vulli Sophie the GiraffeTeether ...,My son loves this toy andfits great in the diaper ...,5.0,"{'and': 1.0, 'right':1.0, 'help': 1.0, 'ju ...",1
Vulli Sophie the GiraffeTeether ...,There really should be alarge warning on the ...,1.0,"{'and': 2.0, 'all': 1.0,'being': 1.0, 'caused': ...",0
Vulli Sophie the GiraffeTeether ...,All the moms in my moms'group got Sophie for ...,5.0,"{'and': 2.0, 'all': 1.0,'love': 1.0, 'have': ...",1
Vulli Sophie the GiraffeTeether ...,I was a little skepticalon whether Sophie was ...,5.0,"{'and': 3.0, 'all': 1.0,'old': 1.0, 'almost': ...",1
Vulli Sophie the GiraffeTeether ...,I have been reading aboutSophie and was going ...,5.0,"{'and': 6.0, 'seven':1.0, 'already': 1.0, ...",1
Vulli Sophie the GiraffeTeether ...,My neice loves her sophieand has spent hours ...,5.0,"{'and': 4.0, 'drooling':1.0, 'additionally': ...",1
Vulli Sophie the GiraffeTeether ...,What a friendly face!And those mesmerizing ...,5.0,"{'and': 3.0, 'chew': 1.0,'sweet': 1.0, 'is': 1.0, ...",1
Vulli Sophie the GiraffeTeether ...,We got this just for myson to chew on instea ...,5.0,"{'chew': 2.0, 'because':1.0, 'just': 2.0, 'fe ...",1
Vulli Sophie the GiraffeTeether ...,This product is without adoubt the best on the ...,5.0,"{'and': 4.0, 'everyone':1.0, 'teething': 1.0, ...",1


In [ ]:
giraffe_reviews.show()

Materializing SFrame

Warning: Skipping column 'word_count'. Unable to show columns of type 'dictionary'; only [int, float, str] can be shown.

Further warnings of unsupported type will be suppressed.

# Quiz Solution


In [ ]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

In [ ]:
products2 = turicreate.SFrame('data/amazon_baby.sframe')
products2['word_count'] = turicreate.text_analytics.count_words(products2['review'])

In [ ]:
#ignore all 3*  reviews
products2 = products2[products2['rating']!= 3]
#positive sentiment = 4-star or 5-star reviews
products2['sentiment'] = products2['rating'] >= 4

In [ ]:
def word_counter(word_count):
    if word in word_count:   
        return word_count[word]
    else:
        return 0
        
for word in selected_words:
    products2[word] = products2['word_count'].apply(word_counter)

In [ ]:
products2

name,review,rating,word_count,sentiment,awesome
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3.0, 'love': 1.0,'it': 3.0, 'highly': ...",1,0.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2.0, 'quilt':1.0, 'it': 1.0, ...",1,0.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'and': 3.0, 'ingenious':1.0, 'love': 2.0, 'is': ...",1,0.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2.0, 'this': 2.0,'all': 2.0, 'love': 1.0, ...",1,0.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2.0, 'cute': 1.0,'help': 2.0, 'habit': ...",1,0.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1.0, 'be': 1.0,'is': 1.0, 'bound': 1.0, ...",1,0.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'and': 2.0, 'all': 1.0,'right': 1.0, 'able': ...",1,0.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1.0, 'fantastic':1.0, 'help': 1.0, 'gi ...",1,0.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'pre': 1.0, 'all': 1.0,'standarad': 1.0, ...",1,0.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'all': 2.0, 'forget':1.0, 'just': 1.0, 'fo ...",1,0.0


In [ ]:
products2['awesome'].sum()

3892.0

In [ ]:
products2['great'].sum()

55791.0

In [ ]:
products2['fantastic'].sum()

1664.0

In [ ]:
products2['amazing'].sum()

2628.0

In [ ]:
products2['love'].sum()


41994.0

In [ ]:
products2['horrible'].sum()

1110

In [ ]:
products2['bad'].sum()

4183

In [ ]:
products2['terrible'].sum()

1146.0

In [ ]:
products2['awful'].sum()

687

In [ ]:
products2['wow'].sum()

425

In [ ]:
products2['hate'].sum()

1107

In [ ]:
train_data2,test_data2 = products2.random_split(.8, seed=0)

In [ ]:
selected_words_model = turicreate.logistic_classifier.create(train_data2,target='sentiment', features=selected_words, validation_set=test_data2)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 11

...

| 4         | 5        | 0.709435     | 0.847708          | 0.846385            |

| 5         | 6        | 0.863885     | 0.847708          | 0.846385            |

| 6         | 7        | 1.030805     | 0.847708          | 0.846385            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

In [ ]:
selected_words_model.coefficients

name,index,class,value,stderr
(intercept),None,1,1.33659138489,0.00892996978766
awesome,None,1,1.13353466603,0.0839964398319
great,None,1,0.86306550012,0.0189550524444
fantastic,None,1,0.885804756881,0.111675912934
amazing,None,1,1.10009331137,0.0995477626047
love,None,1,1.35926886692,0.0280683001521
horrible,None,1,-2.25133523676,0.0802024938879
bad,None,1,-0.991477880065,0.038484286647
terrible,None,1,-2.22366143609,0.0773173620379
awful,None,1,-2.05290820403,0.100997354353


In [ ]:
selected_words_model.coefficients.tail()

name,index,class,value,stderr
great,None,1,0.86306550012,0.0189550524444
fantastic,None,1,0.885804756881,0.111675912934
amazing,None,1,1.10009331137,0.0995477626047
love,None,1,1.35926886692,0.0280683001521
horrible,None,1,-2.25133523676,0.0802024938879
bad,None,1,-0.991477880065,0.038484286647
terrible,None,1,-2.22366143609,0.0773173620379
awful,None,1,-2.05290820403,0.100997354353
wow,None,1,-0.00953823606771,0.160464112247
hate,None,1,-1.34844072225,0.077156986043


In [ ]:
selected_words_model.coefficients['value'].sort()

dtype: float
Rows: 12
[-2.251335236759124, -2.2236614360851554, -2.05290820403138, -1.3484407222463402, -0.9914778800650894, -0.00953823606771224, 0.8630655001195999, 0.8858047568813963, 1.1000933113659914, 1.1335346660341103, 1.3365913848878148, 1.3592688669224757]

In [ ]:
selected_words_model.evaluate(test_data2)

{'accuracy': 0.8463848186404036,
 'auc': 0.6936022046674926,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      1       |        0        |  159  |
 |      0       |        0        |  371  |
 |      0       |        1        |  4957 |
 |      1       |        1        | 27817 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.9157860082304526,
 'log_loss': 0.3962265467087835,
 'precision': 0.8487520595594068,
 'recall': 0.9943165570488991,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+----------------+-----+-------+------+
 | threshold |      fpr       | tpr |   p   |  n   |
 +-----------+----------------+-----+-------+------+
 |    0.0    |      1.0       | 1.0 | 27976 | 5328 |
 |   1e-05  

In [ ]:
sentiment_model.evaluate(test_data)

{'accuracy': 0.9176975738650012,
 'auc': 0.9342357833151299,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        1        |  1397 |
 |      1       |        0        |  1344 |
 |      0       |        0        |  3931 |
 |      1       |        1        | 26632 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.951057941255245,
 'log_loss': 0.3304787187220966,
 'precision': 0.9501587641371436,
 'recall': 0.9519588218472976,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+----------------+----------------+-------+------+
 | threshold |      fpr       |      tpr       |   p   |  n   |
 +-----------+----------------+----------------+-------+------+
 |    0.0    |      1.0       |  

In [ ]:
positive = test_data2['sentiment'].sum()
positive

27976

In [ ]:
len(test_data2)

33304

In [ ]:
diaper_champ_reviews = products[products['name']=='Baby Trend Diaper Champ']

In [ ]:
diaper_champ_reviews

name,review,rating,word_count,sentiment
Baby Trend Diaper Champ,Ok - newsflash. Diapersare just smelly. We've ...,4.0,"{'just': 2.0, 'less':1.0, 'mainly': 1.0, ...",1
Baby Trend Diaper Champ,"My husband and I selectedthe Diaper ""Champ"" ma ...",1.0,"{'selected': 1.0, 'just':1.0, 'less': 1.0, 'wh ...",0
Baby Trend Diaper Champ,Excellent diaper disposalunit. I used it in ...,5.0,"{'control': 1.0, 'am':1.0, 'it': 1.0, 'in': ...",1
Baby Trend Diaper Champ,We love our diaper champ.It is very easy to use ...,5.0,"{'and': 3.0, 'all': 1.0,'love': 1.0, 'just': ...",1
Baby Trend Diaper Champ,Two girlfriends and twofamily members put me ...,5.0,"{'help': 1.0, 'just':1.0, 'keep': 1.0, 'bo ...",1
Baby Trend Diaper Champ,I waited to review thisuntil I saw how it ...,4.0,"{'lysol': 1.0, 'all':1.0, 'perfect': 1.0, ...",1
Baby Trend Diaper Champ,I have had a diaper geniefor almost 4 years since ...,1.0,"{'all': 1.0, 'just': 2.0,'saver': 1.0, 'years': ...",0
Baby Trend Diaper Champ,I originally put thisitem on my baby registry ...,5.0,"{'lysol': 1.0, 'all':4.0, 'feedback': 1.0, ...",1
Baby Trend Diaper Champ,I am so glad I got theDiaper Champ instead of ...,5.0,"{'and': 2.0, 'all': 1.0,'just': 1.0, 'is': 2.0, ...",1
Baby Trend Diaper Champ,We had 2 diaper Genie'sboth given to us as a ...,4.0,"{'rating': 1.0, 'messy':1.0, 'before': 1.0, ...",1


In [ ]:
diaper_champ_reviews = diaper_champ_reviews.sort('predicted_sentiment', ascending=False)

In [ ]:
diaper_champ_reviews

name,review,rating,word_count,sentiment
Baby Trend Diaper Champ,I read a review belowthat can explain exactly ...,4.0,"{'and': 3.0, 'exactly':1.0, 'stink': 1.0, 've': ...",1
Baby Trend Diaper Champ,I have never written areview for Amazon but I ...,5.0,"{'shop': 2.0, 'holds':1.0, 'they': 3.0, 'tip': ...",1
Baby Trend Diaper Champ,I originally put thisitem on my baby registry ...,5.0,"{'lysol': 1.0, 'all':4.0, 'feedback': 1.0, ...",1
Baby Trend Diaper Champ,Baby Luke can turn aclean diaper to a dirty ...,5.0,"{'all': 1.0, 'less': 1.0,'nursery': 1.0, 'smel ...",1
Baby Trend Diaper Champ,Diaper Champ or DiaperGenie? That was my ...,5.0,"{'all': 1.0, 'before':1.0, 'difference': 1.0, ...",1
Baby Trend Diaper Champ,I am one of those super-critical shoppers who ...,5.0,"{'taller': 1.0, 'humid':1.0, 'just': 1.0, 'le ...",1
Baby Trend Diaper Champ,I LOOOVE this diaperpail! Its the easies ...,5.0,"{'just': 1.0, 'over':1.0, 'rweek': 1.0, ...",1
Baby Trend Diaper Champ,"As a first time mother, Iwanted to get the best ...",5.0,"{'and': 2.0, 'love': 1.0,'ve': 1.0, 'figure': ...",1
Baby Trend Diaper Champ,I see that there arecomplaints of stinkiness ...,5.0,"{'all': 1.0, 'over': 1.0,'rotated': 1.0, ...",1
Baby Trend Diaper Champ,I have a 10 year olddaughter and an 8 month ...,5.0,"{'lysol': 1.0, 'all':1.0, 'just': 2.0, 'ov ...",1


In [ ]:
diaper_champ_reviews[0]['review']

"I read a review below that can explain exactly what we experienced. We've had it for 16 months and it has worked wonderful for us. No smells, change it out once a week, easy to clean. Then a diaper snagged this foam material in the head part, so I pulled the rest of the foam out. Big mistake!!! Now it can no loner retain the stinkiness and we're looking for a replacement. Be careful of overloading and never take out that foam piece that is cushioned between pieces. I have figured out that it is key to keeping the stink out."

In [ ]:
selected_words_model.predict(diaper_champ_reviews[0:1], output_type='probability')

dtype: float
Rows: 1
[0.8752178333734397]

In [ ]:
sentiment_model.predict(diaper_champ_reviews[0:1], output_type='probability')

dtype: float
Rows: 1
[0.9999999999895941]

# Sort the Giraffe reviews according to predicted sentiment

In [ ]:
giraffe_reviews = giraffe_reviews.sort('predicted_sentiment', ascending=False)

In [ ]:
giraffe_reviews

name,review,rating,word_count,sentiment
Vulli Sophie the GiraffeTeether ...,"Sophie, oh Sophie, yourtime has come. My ...",5.0,"{'giggles': 1.0, 'all':1.0, '09': 1.0, 'food': ...",1
Vulli Sophie the GiraffeTeether ...,I'll be honest...I boughtthis toy because all the ...,4.0,"{'all': 2.0, 'pops': 1.0,'during': 1.0, ...",1
Vulli Sophie the GiraffeTeether ...,As a mother of 16monthold twins; I bought ...,5.0,"{'cute': 1.0, 'all': 2.0,'just': 1.0, 'able': ...",1
Vulli Sophie the GiraffeTeether ...,We got this littlegiraffe as a gift from a ...,5.0,"{'all': 2.0, 'baby': 1.0,'before': 1.0, 'seems': ...",1
Vulli Sophie the GiraffeTeether ...,"As every mom knows, youalways want to give your ...",5.0,"{'cute': 1.0, 'all': 1.0,'just': 2.0, 'food': ...",1
Vulli Sophie the GiraffeTeether ...,My Mom-in-Law boughtSophie for my son whe ...,5.0,"{'think': 1.0, 'they':1.0, 'just': 1.0, ...",1
Vulli Sophie the GiraffeTeether ...,"My 4 month old son isteething, and I've tried ...",4.0,"{'cute': 1.0, 'all': 2.0,'food': 1.0, 'when': ...",1
Vulli Sophie the GiraffeTeether ...,Let me just start off byaddressing the choking ...,5.0,"{'just': 3.0, 'when':3.0, 'still': 1.0, 'h ...",1
Vulli Sophie the GiraffeTeether ...,I'm not sure why Sophieis such a hit with the ...,4.0,"{'peace': 1.0, 'month':1.0, 'bright': 1.0, ...",1
Vulli Sophie the GiraffeTeether ...,"I admit, I didn't getSophie the Giraffe at ...",4.0,"{'cute': 1.0, 'just':1.0, 'being': 2.0, ...",1


In [ ]:
giraffe_reviews.tail()

name,review,rating,word_count,sentiment
Vulli Sophie the GiraffeTeether ...,My 7 1/2 month oldgrandbaby was sittng up ...,1.0,"{'and': 6.0, 'this': 1.0,'now': 1.0, 'right': ...",0
Vulli Sophie the GiraffeTeether ...,I was so looking forwardto getting this for my ...,1.0,"{'things': 1.0, 'just':2.0, 'over': 1.0, ...",0
Vulli Sophie the GiraffeTeether ...,Received the product andsmells like cheap rub ...,1.0,"{'and': 1.0, 'money':2.0, 'over': 1.0, ...",0
Vulli Sophie the GiraffeTeether ...,I got one of these as ashowe gift that my baby ...,1.0,"{'all': 2.0, 'press':1.0, 'shape': 3.0, ...",0
Vulli Sophie the GiraffeTeether ...,When we first got it Ithought did my ...,2.0,"{'and': 2.0, 'cute': 1.0,'btw': 1.0, 'because': ...",0
Vulli Sophie the GiraffeTeether ...,When I received this thepaint was peeling off in ...,1.0,"{'and': 2.0, 'everyone':1.0, 'help': 1.0, 'gi ...",0
Vulli Sophie the GiraffeTeether ...,"""Sophie the Giraffe"" hastested positive for ...",1.0,"{'saying': 1.0,'chances': 1.0, 'caus ...",0
Vulli Sophie the GiraffeTeether ...,"My son (now 2.5) LOVEDhis Sophie, and I bought ...",1.0,"{'all': 1.0, 'just': 1.0,'over': 2.0, 'not': 3.0, ...",0
Vulli Sophie the GiraffeTeether ...,I wanted to love thisproduct and was excited ...,1.0,"{'just': 1.0, 'children':1.0, 'hmmm': 1.0, ...",0
Vulli Sophie the GiraffeTeether ...,This children's toy isnostalgic and very cute. ...,1.0,"{'refund': 1.0, 'enjoy':1.0, 'do': 1.0, 'taste': ...",0


## Show the most positive reviews

In [ ]:
giraffe_reviews[0]['review']

"Sophie, oh Sophie, your time has come. My granddaughter, Violet is 5 months old and starting to teeth. What joy little Sophie brings to Violet. Sophie is made of a very pliable rubber that is sturdy but not tough. It is quite easy for Violet to twist Sophie into unheard of positions to get Sophie into her mouth. The little nose and hooves fit perfectly into small mouths, and the drooling has purpose. The paint on Sophie is food quality.Sophie was born in 1961 in France. The maker had wondered why there was nothing available for babies and made Sophie from the finest rubber, phthalate-free on St Sophie's Day, thus the name was born. Since that time millions of Sophie's populate the world. She is soft and for babies little hands easy to grasp. Violet especially loves the bumpy head and horns of Sophie. Sophie has a long neck that easy to grasp and twist. She has lovely, sizable spots that attract Violet's attention. Sophie has happy little squeaks that bring squeals of delight from Viol

In [ ]:
giraffe_reviews[1]['review']

'I\'ll be honest...I bought this toy because all the hip parents seem to have one too and I wanted to be a part of the "hip parent" crowd. The price-tag was somewhat of a deterent but I prevailed and purchased this teether for my daughter.At first, Lily didn\'t know what to make of of Sophie and showed little interest in the polka-dotted creature. I continued to introduce Lily to Sophie and kept the toy in the carrier so that it was on-hand during transitions. Eventually, Lily discovered what a wonderful experience it was to gnaw on the hooves and ears and these two have never been far apart since.Lily really enjoys gumming all the different parts of Sophie like no other teether we have. The size of the toy is great as it is somewhat substantial and so easy for a little one to grasp and hold onto. Lily really enjoys hearing Sophie squeak and will smile whenever Sophie makes a noise or pops her head up from Mommy\'s lap to say hello.People have stopped and commented on Sophie and to the

# Most negative reivews

In [ ]:
giraffe_reviews[-1]['review']

"This children's toy is nostalgic and very cute. However, there is a distinct rubber smell and a very odd taste, yes I tried it, that my baby did not enjoy. Also, if it is soiled it is extremely difficult to clean as the rubber is a kind of porus material and does not clean well. The final thing is the squeaking device inside which stopped working after the first couple of days. I returned this item feeling I had overpaid for a toy that was defective and did not meet my expectations. Please do not be swayed by the cute packaging and hype surounding it as I was. One more thing, I was given a full refund from Amazon without any problem."

In [ ]:
giraffe_reviews[-2]['review']

'I wanted to love this product and was excited to buy it when I became pregnant but am now hesitant to let my baby use it after reading about the recall in Europe. Apparently, as I understand it, their toxin standards of measurement are lower than ours so they have not been recalled here (apparently we are OK with low levels of nitrates in the toys our children put in their mouths, but Europeans are not...hmmm)...Be that as it may, toxins registering even CLOSE to a dangerous level made me nervous about using. After digging around online I did discover that the company claims to have changed the product after a certain date and lists manufacturing codes so you can check yours (those listed were made after a certain date and are said to be safer). Sadly mine was not made after the &#34;improved&#34; date but I could not return it because there was no formal recall in our country. I considered returning it and hunting for one with an approved manufacturing date but man that was just too 